## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-01-18-50-10 +0000,nyt,"After Days of Protests, Tanzania’s President I...",https://www.nytimes.com/2025/11/01/world/afric...
1,2025-11-01-18-47-54 +0000,nypost,Ukrainian intelligence identified locations of...,https://nypost.com/2025/11/01/world-news/ukrai...
2,2025-11-01-18-39-38 +0000,bbc,"Egypt's Grand Museum opens, displaying Tutankh...",https://www.bbc.com/news/articles/ckg4q403rpzo...
3,2025-11-01-18-30-14 +0000,nypost,"UPenn investigating vulgar, fake emails with s...",https://nypost.com/2025/11/01/us-news/upenn-in...
4,2025-11-01-18-26-24 +0000,bbc,US judges say Trump administration must contin...,https://www.bbc.com/news/articles/cr433x9zqq4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,26
119,new,15
376,he,13
5,election,12
49,shutdown,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
180,2025-10-31-23-04-00 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...,68
218,2025-10-31-20-26-00 +0000,wsj,The federal government pays out $100 billion a...,https://www.wsj.com/politics/policy/what-to-kn...,67
116,2025-11-01-08-43-08 +0000,nyt,Carney Says He Apologized to Trump Over Ad Rev...,https://www.nytimes.com/2025/11/01/world/canad...,64
4,2025-11-01-18-26-24 +0000,bbc,US judges say Trump administration must contin...,https://www.bbc.com/news/articles/cr433x9zqq4o...,64
47,2025-11-01-14-00-00 +0000,wsj,Canadian PM Carney Says He Apologized to Trump...,https://www.wsj.com/world/canadian-pm-carney-s...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
180,68,2025-10-31-23-04-00 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...
218,56,2025-10-31-20-26-00 +0000,wsj,The federal government pays out $100 billion a...,https://www.wsj.com/politics/policy/what-to-kn...
59,54,2025-11-01-12-55-44 +0000,cbc,"'This is not a game,' Carney says as he signal...",https://www.cbc.ca/news/politics/carney-budget...
173,49,2025-10-31-23-24-00 +0000,wsj,The White House is imposing new restrictions o...,https://www.wsj.com/politics/policy/white-hous...
53,37,2025-11-01-13-14-15 +0000,nypost,"Andrew Cuomo, Curtis Sliwa closing gap on soci...",https://nypost.com/2025/11/01/us-news/andrew-c...
160,34,2025-11-01-00-09-02 +0000,nypost,JD Vance dresses up like meme-version of himse...,https://nypost.com/2025/10/31/us-news/jd-vance...
10,29,2025-11-01-17-55-30 +0000,nypost,Atlantic City Mayor Marty Small Sr. expected t...,https://nypost.com/2025/11/01/us-news/atlantic...
29,27,2025-11-01-15-26-45 +0000,nyt,"In One Jamaican Town, Hurricane Melissa Left B...",https://www.nytimes.com/2025/10/31/world/ameri...
141,27,2025-11-01-03-00-00 +0000,wsj,Progressives in Maine are excited by Graham Pl...,https://www.wsj.com/politics/elections/graham-...
158,26,2025-11-01-00-45-49 +0000,nypost,Nancy Mace allegedly berated cops in profanity...,https://nypost.com/2025/10/31/us-news/nancy-ma...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
